# Image Classification

Reads the processed inout data.

Creates the model.

Trains the model.

Tests the model.

---

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow import keras
from keras.utils import img_to_array
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D

from sklearn.model_selection import train_test_split
from PIL import Image

## Data Reading/Processing

In [2]:
df = pd.read_csv('data.csv')
df.groupby('Midjourney').value_counts()
df.describe()

,Midjourney,Wikiart
count,9009.00000,9009.00000
mean,0.50272,0.49728
std,0.50002,0.50002
min,0.00000,0.00000
25%,0.00000,0.00000
50%,1.00000,0.00000
75%,1.00000,1.00000
max,1.00000,1.00000


In [3]:
X = df[['img']]
# y = df.drop(columns=['img'])
y = df['Midjourney']
y.value_counts()

1    4529
0    4480
Name: Midjourney, dtype: int64

Converts the images to a format readable by the model:

In [4]:
all_images = []
for i, image_path in enumerate(X['img']):
    img = Image.open(image_path)
    # print(img.size)
    array = tf.keras.utils.img_to_array(img)
    
    if (array.size != 196608):
        print(image_path)
        y = y.drop(index=i)
        continue
    all_images.append(array)
X = all_images

In [5]:
X = np.array(X)
y = np.array(y)
y

array([0, 0, 0, ..., 1, 1, 1])

In [6]:
X[0].shape

(256, 256, 3)

Splits the data into training and testing segments:

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=337)
X_train /= 255.0
X_test /= 255.0

In [8]:
y_train

array([0, 0, 0, ..., 0, 0, 1])

In [9]:
input_shape = (256, 256, 3)

## Model Creation

In [11]:
model = Sequential()

model.add(Conv2D(filters=32,
                  kernel_size=(3, 3),  
                  activation='relu',
                  input_shape=input_shape))  
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 254, 254, 32)      896       
                                                                 
 conv2d_3 (Conv2D)           (None, 252, 252, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 126, 126, 64)     0         
 2D)                                                             
                                                                 
 dropout_2 (Dropout)         (None, 126, 126, 64)      0         
                                                                 
 flatten_1 (Flatten)         (None, 1016064)           0         
                                                                 
 dense_2 (Dense)             (None, 128)               130056320 
                                                      

In [12]:
model.compile(loss=tf.keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

## Model Training

In [13]:
model.fit(X_train, y_train, batch_size=64, epochs=3, verbose=1)

Epoch 1/3
127/127 [==============================] - 713s 6s/step - loss: 1.5321 - accuracy: 0.7828
Epoch 2/3
127/127 [==============================] - 692s 5s/step - loss: 0.2690 - accuracy: 0.9018
Epoch 3/3
127/127 [==============================] - 648s 5s/step - loss: 0.1644 - accuracy: 0.9414


## Model Testing

In [14]:
results = model.evaluate(X_test, y_test, batch_size=64)

15/15 [==============================] - 15s 940ms/step - loss: 0.2825 - accuracy: 0.9134


In [71]:
predictions = model.predict(X_test[:10])
predictions

1/1 [==============================] - 0s 203ms/step


array([[0.9784825 ],
       [0.994701  ],
       [0.97999996],
       [0.84798586],
       [0.99917567],
       [0.02512486],
       [0.7489047 ],
       [0.2664383 ],
       [0.4959675 ],
       [0.9948937 ]], dtype=float32)

In [152]:
model = Sequential()

model.add(Conv2D(filters=32,
                  kernel_size=(3, 3),  
                  activation='relu',
                  input_shape=input_shape))  
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(2, activation='softmax'))

# print model summary
model.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_30 (Conv2D)          (None, 254, 254, 32)      896       
                                                                 
 conv2d_31 (Conv2D)          (None, 252, 252, 64)      18496     
                                                                 
 max_pooling2d_19 (MaxPoolin  (None, 126, 126, 64)     0         
 g2D)                                                            
                                                                 
 dropout_4 (Dropout)         (None, 126, 126, 64)      0         
                                                                 
 flatten_11 (Flatten)        (None, 1016064)           0         
                                                                 
 dense_21 (Dense)            (None, 128)               130056320 
                                                     